In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

# Load the dataset
df = pd.read_csv('Alphabets_data.csv')

# Explore the dataset
print(df.head())
print(df.info())
print(df.describe())

# Handle missing values if any
df.fillna(df.mean(), inplace=True)

# Separate features and target
X = df.drop('letter', axis=1)
y = df['letter']

# Encode the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Define a function to create the ANN model
def create_model(optimizer='adam', activation='relu', neurons=32):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(len(np.unique(y)), activation='softmax'))  # Adjust the output layer for multi-class classification
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create and compile the model
model = create_model()
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Hyperparameter Tuning
model = KerasClassifier(model=create_model, verbose=0)
param_grid = {
    'model__neurons': [16, 32, 64],
    'model__activation': ['relu', 'tanh'],
    'model__optimizer': ['adam', 'rmsprop'],
    'fit__epochs': [50, 100],
    'fit__batch_size': [10, 20]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Summarize the results
print(f'Best Score: {grid_result.best_score_}')
print(f'Best Params: {grid_result.best_params_}')

# Evaluate the tuned model
best_model = grid_result.best_estimator_
y_pred_tuned = best_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
precision_tuned = precision_score(y_test, y_pred_tuned, average='weighted')
recall_tuned = recall_score(y_test, y_pred_tuned, average='weighted')
f1_tuned = f1_score(y_test, y_pred_tuned, average='weighted')

print(f'Tuned Accuracy: {accuracy_tuned}')
print(f'Tuned Precision: {precision_tuned}')
print(f'Tuned Recall: {recall_tuned}')
print(f'Tuned F1 Score: {f1_tuned}')


  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbo

C:\Users\nikhi\AppData\Local\Temp\ipykernel_10232\2621187705.py:22: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)
C:\Users\nikhi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,402 (5.48 KB)

 Trainable params: 1,402 (5.48 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3305 - loss: 2.4070 - val_accuracy: 0.6803 - val_loss: 1.2159
Epoch 2/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7132 - loss: 1.0709 - val_accuracy: 0.7494 - val_loss: 0.9258
Epoch 3/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 957us/step - accuracy: 0.7620 - loss: 0.8515 - val_accuracy: 0.7738 - val_loss: 0.7982
Epoch 4/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 927us/step - accuracy: 0.7921 - loss: 0.7306 - val_accuracy: 0.7894 - val_loss: 0.7201
Epoch 5/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8092 - loss: 0.6633 - val_accuracy: 0.8041 - val_loss: 0.6660
Epoch 6/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8153 - loss: 0.6151 - val_accuracy: 0.8181 - val_loss: 0.6163
Epoch 7/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 984us/step - accuracy: 0.8302 - loss: 0.5826 - val_accuracy: 0.8234 - val_loss: 0.5854
Epoch 8/50
1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 990us/step - accuracy: 0.8391 - loss: 0

1280/1280 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9266 - loss: 0.2351 - val_accuracy: 0.8997 - val_loss: 0.3227
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step
Accuracy: 0.89625
Precision: 0.8981184727544047
Recall: 0.89625
F1 Score: 0.8963704296497437
